## importação dos pacotes


In [1]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import traceback
from datetime import date


## variaveis de controle

In [2]:
server_target = 'rgprd-sqlsrv-prd01.database.windows.net'
database_master = 'master'
username_server_target = 'implanta.net'
password_target = 'M@st3rP0w3r@zur3Prd'
driver= '{ODBC Driver 17 for SQL Server}'

server_source ='DS10\MSSQLSERVER2017'
database_source ="Elmah"
user_source ="wesley"
password_source ="M@st3rP0w3r@zur3Dev"

dados_df = pd.DataFrame()

data_atual = date.today()



## Utils


In [3]:
def get_conection(driver,server,database,username,password):
    _conexao = PDB.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    return _conexao  

## Motodo para executar o select em cada banco de dados

In [4]:
def executa_sql_get_elmah(database_name):
    dados = pd.DataFrame()
    try:
        _con = get_conection(driver,server_target,database_name,username_server_target,password_target)
        
        query =" EXEC dbo.GetElmahErrosOnAplication"
        

        SQL_exec_procedure = pd.read_sql_query(query,_con)

        dados = pd.DataFrame(SQL_exec_procedure, columns=['ErrorId','Application','Host','Type','Source','Message','User','StatusCode', 
         'TimeUtc','AllXml','ServerName'])
         
        ##dados.set_index('ErrorId',inplace=True)
        _con.close() 
        return dados

    except TypeError:
        _con.close()
    return dados


    

## executa teste

In [9]:
dados =  executa_sql_get_elmah('crea-sp.implanta.net.br')

In [6]:
dados.count()

ErrorId        1123
Application    1123
Host           1123
Type           1123
Source         1123
Message        1123
User           1123
StatusCode     1123
TimeUtc        1123
AllXml         1123
ServerName     1123
dtype: int64

In [10]:

dados = dados[~dados['Message'].str.startswith("Execution")] # remove os erros do Hangfire

dados = dados[~dados['Message'].str.startswith("Configuração do parcelamento não informada.")] # Remove os erros de migração

dados = dados[~dados['Message'].str.startswith("payment_type")] # Remove as mensagens de cartão de creditos

dados = dados[~dados['Message'].str.contains("order_number")] # Remove as mensagens de cartão de creditos






,ErrorId,Application,Host,Type,Source,Message,User,StatusCode,TimeUtc,AllXml,ServerName
1,C5F0B2BD-03D5-4DAA-9C94-C56AD231C6D0,Logon,PRD-CREA-1,System.Data.SqlClient.SqlException,.Net SqlClient Data Provider,Transaction (Process ID 114) was deadlocked on...,,0,2021-02-08 11:40:19.423,"<error\r\n application=""Logon""\r\n host=""PRD...",crea-sp.implanta.net.br
2,B990F57F-50B7-4747-8E7C-61EF0147157B,Logon,PRD-CREA-1,System.Transactions.TransactionAbortedException,System.Transactions,The transaction has aborted.,,500,2021-02-08 11:40:19.500,"<error\r\n application=""Logon""\r\n host=""PRD...",crea-sp.implanta.net.br
3,B925DBC4-0873-4511-9BF7-EEBFDC899BAE,Siscont,PRD-CREA-1,System.NullReferenceException,App_Web_uskiekn0,Object reference not set to an instance of an ...,VANDETEJO,500,2021-02-08 19:08:34.657,"<error\r\n application=""Siscont""\r\n host=""P...",crea-sp.implanta.net.br
4,C379B743-1180-417C-9D8B-A16A66CB95B3,Logon,PRD-CREA-1,System.Data.SqlClient.SqlException,.Net SqlClient Data Provider,Transaction (Process ID 424) was deadlocked on...,,0,2021-02-09 10:30:04.083,"<error\r\n application=""Logon""\r\n host=""PRD...",crea-sp.implanta.net.br
7,A046437E-4ACF-4862-A217-0197437FAFA3,Siscont,PRD-CREA-1,System.ArgumentException,Newtonsoft.Json,Could not cast or convert from System.String t...,LENIRADM,500,2021-02-08 11:47:19.217,"<error\r\n application=""Siscont""\r\n host=""P...",crea-sp.implanta.net.br


## Conecta no servidor target e source 

In [11]:
database = 'master'

conexao =get_conection(driver,server_target,database,username_server_target,password_target)

target_conection =get_conection(driver,server_source,database_source,user_source,password_source)
 

## Testa a Conexão com Source

In [12]:
cursor = target_conection.cursor()


cursor.execute("SELECT @@Version") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

cursor.close()

Microsoft SQL Server 2017 (RTM) - 14.0.1000.169 (X64) 
	Aug 22 2017 17:04:49 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 18362: ) (Hypervisor)



## Monta a query para ler todos so bancos de dados

In [13]:
SQL_Query = pd.read_sql_query(
'''SELECT  D.database_id,D.name FROM  sys.databases AS D
 order by D.database_id''', conexao)

##Monta o DataFrame

In [14]:
databases = pd.DataFrame(SQL_Query, columns=['database_id','name'])

In [15]:
databases.count()

database_id    509
name           509
dtype: int64

## Executa um filtro para executar as querys somente nos databases selecionados

In [16]:
bancos_filtrados = databases[~databases['name'].str.contains("master|Manager|ajuda|Conversor|conversor|Copy|DNE|automationjobs|Configuracao|rglab|ESPELHO|apresentacao|COPY|Homologacao|EXCLUIR|202|PreMig|conversor|treinamento|patrimonio")]

In [17]:
bancos_filtrados.count()

database_id    411
name           411
dtype: int64

In [24]:
bancos_filtrados.sample(10)

,database_id,name
280,292,coren-rs.implanta.net.br
328,341,crf-se.implanta.net.br
107,111,coren-go.implanta.net.br
310,322,crp-se.implanta.net.br
302,314,crp-rs.implanta.net.br
224,236,crfa-mg.implanta.net.br
243,255,crfa-sp.implanta.net.br
189,195,crmv-pa.implanta.net.br
88,92,cau-pi.implanta.net.br
270,282,crf-mg.implanta.net.br


##Executa um for each para cada coluna datetime2

In [18]:
target_conection =get_conection(driver,server_source,database_source,user_source,password_source)

In [26]:
def insertElmah_intoTarget(data,target_conection):
    cursor = target_conection.cursor()
    
    for i,row in data.iterrows():
        sql =""
        sql = ("INSERT INTO Staging.ELMAH_Error(ErrorId,Application,Host,Type,Source,Message,[User],StatusCode,TimeUtc,AllXml,ServerName)"
        "SELECT '{0}', '{1}' ,'{2}','{3}','{4}','{5}','{6}',{7},'{8}',CAST('{9}' AS XML), '{10}' "
        " WHERE NOT EXISTS(SELECT 1 FROM Staging.ELMAH_Error AS EE WHERE EE.ErrorId ='{0}' )").format(row["ErrorId"],
        row["Application"],row["Host"],row["Type"],row['Source'],row["Message"].replace("'","''"),row["User"],
        row["StatusCode"],row["TimeUtc"].strftime("%Y-%m-%d %H:%M:%S"),row["AllXml"].replace("'","''"),row["ServerName"])
        cursor.execute(sql)
         
    # print(sql)
# the connection is not autocommitted by default, so we must commit to save our # changes 
    target_conection.commit()
    #return sql


In [23]:
insertElmah_intoTarget(dados,target_conection)

In [27]:
try:
    database =""
    for i, linha in bancos_filtrados.sort_values("name").iterrows():
        database =linha['name']
        retorno = executa_sql_get_elmah(linha['name'])
        
        # Executa o tratamento dos dados
        retorno = retorno[~retorno['Message'].str.startswith("Execution")] # remove os erros do Hangfire

        retorno = retorno[~retorno['Message'].str.startswith("Configuração do parcelamento não informada.")] # Remove os erros de migração

        retorno = retorno[~retorno['Message'].str.startswith("payment_type")] # Remove as mensagens de cartão de creditos

        retorno = retorno[~retorno['Message'].str.contains("order_number")] # Remove as mensagens de cartão de creditos

        retorno = retorno[~retorno['Message'].str.contains("Monitoramento")] # Remove as mensagens de cartão de creditos

        retorno = retorno[~retorno['Message'].str.contains("amount")] # Remove as mensagens de cartão de creditos

        







        print(database,"Total:",retorno["ErrorId"].count())
        comando = insertElmah_intoTarget(retorno,target_conection)

except BaseException as ex:
    # Get current system exception
    ex_type, ex_value, ex_traceback = sys.exc_info()

    # Extract unformatter stack traces as tuples
    trace_back = traceback.extract_tb(ex_traceback)

    # Format stacktrace
    stack_trace = list()

    for trace in trace_back:
        stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

    print("Exception type : %s " % ex_type.__name__)
    print("Exception message : %s" %ex_value)
    print("Stack trace : %s" %stack_trace)
    print("database : %s" %database)
    print(linha['name'],tableName,coluna,idregistro)         
            
            
        





caa-ma.implanta.net.br Total: 0
cau-ac.implanta.net.br Total: 15
cau-al.implanta.net.br Total: 10
cau-am.implanta.net.br Total: 4
cau-ap.implanta.net.br Total: 1
cau-ba.implanta.net.br Total: 4
cau-br.implanta.net.br Total: 35
cau-ce.implanta.net.br Total: 5
cau-df.implanta.net.br Total: 19
cau-es.implanta.net.br Total: 7
cau-go.implanta.net.br Total: 7
cau-ma.implanta.net.br Total: 29
cau-mg.implanta.net.br Total: 30
cau-ms.implanta.net.br Total: 4
cau-mt.implanta.net.br Total: 27
cau-pa.implanta.net.br Total: 10
cau-pb.implanta.net.br Total: 15
cau-pe.implanta.net.br Total: 9
cau-pi.implanta.net.br Total: 4
cau-pr.implanta.net.br Total: 30
cau-rj.implanta.net.br Total: 8
cau-rn.implanta.net.br Total: 2
cau-ro.implanta.net.br Total: 11
cau-rr.implanta.net.br Total: 1
cau-rs.implanta.net.br Total: 20
cau-sc.implanta.net.br Total: 43
cau-se.implanta.net.br Total: 19
cau-sp.implanta.net.br Total: 47
cau-to.implanta.net.br Total: 0
cfa-br.implanta.net.br Total: 0
cfbm-br.implanta.net.br T

In [28]:
target_conection.close()
conexao.close()